# Getting started

The `pivmetalib` implements the classes for the PIVmeta ontology. `ontolutils` implements many (but not all) classes for various ontologies and vocabularies used in the ontology ["pivmeta"](https://matthiasprobst.github.io/pivmeta/).

By means of `pivmetalib` you are able to instantiate objects of the ontology `pivmeta` (and the related ones via `ontolutils`).

Let's find out, what that means for the description of a Person (A person can be described by the PROV namespace: https://www.w3.org/ns/prov)

In [1]:
from ontolutils.ex import prov

In [2]:
creator = prov.Person(
    lastName='Okamoto',
    mbox="okamoto@tokai.t.u-tokyo.ac.jp"
)
creator

Person(id=_:N12f35d4d73614b528e5953faff7f823e, mbox=okamoto@tokai.t.u-tokyo.ac.jp, lastName=Okamoto)

Most importantly, dumping the data to a JSON-LD string is most practical and useful for use in data description. For this call `model_dump_jsonld()`:

In [3]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#"
    },
    "@type": "prov:Person",
    "foaf:mbox": "okamoto@tokai.t.u-tokyo.ac.jp",
    "foaf:lastName": "Okamoto",
    "@id": "_:N12f35d4d73614b528e5953faff7f823e"
}


**What's behind the class:** Each class is inherited from the `ontolutils.Thing`, which again is a `pydantic.BaseModel` subclass. The package [`pydantic`](https://docs.pydantic.dev/latest/) allows validating the attributes of a class. This means, that passing wrong types to the person, will lead to an error:

In [4]:
import pydantic

In [5]:
try:
    prov.Person(lastName='John', mbox='email.com')
except pydantic.ValidationError as e:
    print(e)

1 validation error for Person
mbox
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='email.com', input_type=str]


**Extra fields**

Generally, it is possible to add any additional fields, like age, for example. The field "age" will wil appear in the JSON-LD string, however without a *prefix*:

In [6]:
creator = prov.Person(
    firtName='John',
    lastName='Doe',
    age=34
)
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "age": 34,
    "@id": "_:Nea28a48b20464054826b361d70b73814"
}


To change this, we can update the missing URI at all times:

In [7]:
creator.urirefs['age'] = 'foaf:age'

In [8]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "foaf:age": 34,
    "@id": "_:Nea28a48b20464054826b361d70b73814"
}


## Working with the classes

There are two importing things, you can do with the classes:
- **Dumping**: This means exporting the content to various formats (dictionary, json-ld strings, ...)
- **querying**: Finding data of a class in a JSON-LD file

### Dumping:
The most basic form of "dumping" is exporting the class instance to a dictionary:

In [9]:
creator.model_dump()

{'id': '_:Nea28a48b20464054826b361d70b73814',
 'label': None,
 'about': None,
 'relation': None,
 'closeMatch': None,
 'exactMatch': None,
 'mbox': None,
 'firstName': None,
 'lastName': 'Doe',
 'orcidId': None,
 'affiliation': None,
 'firtName': 'John',
 'age': 34}

The `pivmetalib` adds also `dump_jsonld` to the classes and allows to create a JSON-LD string, which can be saved to a file, too:

In [10]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "foaf:age": 34,
    "@id": "_:Nea28a48b20464054826b361d70b73814"
}


**Save JSON-LD file**

In [11]:
import pivmetalib

In [12]:
# save to file:
with open('john.jsonld', 'w') as f:
    creator.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT})

Let's create another person and save both to the file. For this we will need to "merge" both JSON-LD strings. There is a helper function for this in the `utils` module: 

In [13]:
other_person = prov.Person(first_name='Lisa')

In [14]:
from pivmetalib.jsonld import merge

In [15]:
# save to file:
with open('creator.jsonld', 'w') as f:
    f.write(merge([creator.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT}),
                   other_person.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT})]))

### Querying

We can identify data within a JSON-LD file by calling `query` and providing the class we want to find. The method applies a SPARQL query ad returns an instance of the provided object:

In [16]:
from ontolutils import query

In [17]:
persons = query(cls=prov.Person, source='creator.jsonld', context={"@import": pivmetalib.CONTEXT})
persons

[Person(id=_:Nea28a48b20464054826b361d70b73814, lastName=Doe, firtName=John, age=34),
 Person(id=_:Ne5e5e5050db64527bae67d30c5f76bab, firstName=Lisa)]

Likewise, you can call `from_jsonld` from the Person object:

In [18]:
persons = prov.Person.from_jsonld(source='creator.jsonld', context={"@import": pivmetalib.CONTEXT})
persons

[Person(id=_:Nea28a48b20464054826b361d70b73814, lastName=Doe, firtName=John, age=34),
 Person(id=_:Ne5e5e5050db64527bae67d30c5f76bab, firstName=Lisa)]

As expected, we find both persons:

In [19]:
import rdflib

In [20]:
g = rdflib.Graph().parse(source='creator.jsonld', context=pivmetalib.CONTEXT)

In [21]:
for s,p,o in g:
    print(s, o)

Nea28a48b20464054826b361d70b73814 John
Nea28a48b20464054826b361d70b73814 34
Nea28a48b20464054826b361d70b73814 http://www.w3.org/ns/prov#Person
Nea28a48b20464054826b361d70b73814 Doe
Ne5e5e5050db64527bae67d30c5f76bab http://www.w3.org/ns/prov#Person
Ne5e5e5050db64527bae67d30c5f76bab Lisa


In [22]:
qs = """PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX local: <http://example.com/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT *
WHERE {
    ?id a prov:Person .
    ?id ?p ?o .
}"""

In [23]:
for r in g.query(qs):
    print(r)

(rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.BNode('Nea28a48b20464054826b361d70b73814'), rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'))
(rdflib.term.URIRef('http://xmlns.com/foaf/0.1/lastName'), rdflib.term.BNode('Nea28a48b20464054826b361d70b73814'), rdflib.term.Literal('Doe'))
(rdflib.term.URIRef('https://matthiasprobst.github.io/pivmeta#firtName'), rdflib.term.BNode('Nea28a48b20464054826b361d70b73814'), rdflib.term.Literal('John'))
(rdflib.term.URIRef('http://xmlns.com/foaf/0.1/age'), rdflib.term.BNode('Nea28a48b20464054826b361d70b73814'), rdflib.term.Literal('34', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.BNode('Ne5e5e5050db64527bae67d30c5f76bab'), rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'))
(rdflib.term.URIRef('http://xmlns.com/foaf/0.1/firstName'), rdflib.term.BNode('Ne5e5e5050db64527bae67d30c5f76bab'), rdfl

In [24]:
for person in persons:
    print(person.model_dump(exclude_none=True))

{'id': '_:Nea28a48b20464054826b361d70b73814', 'lastName': 'Doe', 'firtName': 'John', 'age': '34'}
{'id': '_:Ne5e5e5050db64527bae67d30c5f76bab', 'firstName': 'Lisa'}
